In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.colors as mcolors
import sys
import os
import cv2
import torch
import torch.nn as nn
from collections import OrderedDict
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms
from scipy.ndimage import zoom

In [ ]:
def read_kitti_point_cloud(file_path):
    # Read point cloud data from the KITTI dataset .bin file
    return np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)[:, :3]  # Extract x, y, z coordinates

def voxelization(point_cloud, voxel_size):
    # Determine the grid size and resolution
    min_coords = np.min(point_cloud, axis=0)
    max_coords = np.max(point_cloud, axis=0)
    grid_size = np.ceil((max_coords - min_coords) / voxel_size).astype(int)

    # Create an empty voxel grid
    voxel_grid = np.zeros(grid_size, dtype=int)

    # Assign points to voxels
    voxel_indices = ((point_cloud - min_coords) / voxel_size).astype(int)
    for voxel_idx in voxel_indices:
        voxel_grid[tuple(voxel_idx)] += 1  # Increment voxel count

    return voxel_grid

if __name__ == "__main__":
    # Replace 'path_to_kitti_point_cloud.bin' with the actual path to your KITTI dataset .bin file
    point_cloud_data = read_kitti_point_cloud('/content/drive/MyDrive/Occlusion_analysis/YOLO3D-YOLOv4-PyTorch/Voxelnet/dataset/kitti/training/velodyne/000000.bin')

    # Voxelization parameters (you can adjust these as needed)
    voxel_size = 0.1  # Voxel size in meters

    # Perform voxelization
    voxel_grid = voxelization(point_cloud_data, voxel_size)

    # Visualize the voxel grid
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Get the indices of the occupied voxels
    x, y, z = np.where(voxel_grid > 0)

    # Plot the occupied voxels
    ax.scatter(x, y, z, c='b', marker='.', s=1)

    # Set axis labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    plt.show()

    # Print the size and shape of the voxel grid
    print("Voxel Grid Shape:", voxel_grid.shape)
    print("Number of Occupied Voxels:", np.sum(voxel_grid > 0))

In [ ]:
def voxelization_with_density(point_cloud, voxel_size):
    # Find the minimum and maximum coordinates of the point cloud
    min_coords = np.min(point_cloud, axis=0)
    max_coords = np.max(point_cloud, axis=0)

    # Calculate the number of voxels in each dimension
    num_voxels_x = int(np.ceil((max_coords[0] - min_coords[0]) / voxel_size))
    num_voxels_y = int(np.ceil((max_coords[1] - min_coords[1]) / voxel_size))
    num_voxels_z = int(np.ceil((max_coords[2] - min_coords[2]) / voxel_size))

    # Initialize the voxel grid and point density grid
    voxel_grid = np.zeros((num_voxels_x, num_voxels_y, num_voxels_z), dtype=np.int32)
    point_density_grid = np.zeros((num_voxels_x, num_voxels_y, num_voxels_z), dtype=np.float32)

    # Loop through each point in the point cloud and populate the voxel grid
    cnt =0
    for point in point_cloud:
        cnt = cnt +1
        # Calculate the voxel index for the current point
        voxel_x = int(np.floor((point[0] - min_coords[0]) / voxel_size))
        voxel_y = int(np.floor((point[1] - min_coords[1]) / voxel_size))
        voxel_z = int(np.floor((point[2] - min_coords[2]) / voxel_size))

        # Check if the voxel indices are within bounds
        if voxel_x >= 0 and voxel_x < num_voxels_x and voxel_y >= 0 and voxel_y < num_voxels_y and voxel_z >= 0 and voxel_z < num_voxels_z:
            voxel_grid[voxel_x, voxel_y, voxel_z] += 1
            point_density_grid[voxel_x, voxel_y, voxel_z] += 1
        else:
            print("Warning: Point outside voxel grid bounds.")

    return voxel_grid, point_density_grid

# Example usage:
# Replace 'path_to_kitti_point_cloud.bin' with the actual path to your KITTI dataset .bin file
point_cloud_data = read_kitti_point_cloud('/content/drive/MyDrive/Occlusion_analysis/YOLO3D-YOLOv4-PyTorch/Voxelnet/dataset/kitti/training/velodyne/000016.bin')

# Voxelization parameters (you can adjust these as needed)
voxel_size = 0.1  # Voxel size in meters

# Perform voxelization with point density
voxel_grid, point_density_grid = voxelization_with_density(point_cloud_data, voxel_size)
